In [1]:
import pandas as pd

In [49]:
apikey = '5737e060-0f84-4251-902f-0716332388f1'

In [50]:
import requests
from geopy.distance import geodesic

def fetch_coordinates(apikey, place):
    base_url = "https://geocode-maps.yandex.ru/1.x"
    params = {"geocode": place, "apikey": apikey, "format": "json"}
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    places_found = response.json()['response']['GeoObjectCollection']['featureMember']
    most_relevant = places_found[0]
    lon, lat = most_relevant['GeoObject']['Point']['pos'].split(" ")
    return lon, lat

In [2]:
dataset = pd.read_csv('dataset.csv', index_col=0)

In [80]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def search(needed, center_city):
    frame = pd.DataFrame(columns=dataset.columns.tolist())
    center = fetch_coordinates(apikey, center_city)
    needed_lst = needed.split(' ')
    for word in needed_lst:
        p = morph.parse(word)[0]
        slovo = p.normal_form
        for i in range(0, len(dataset.title.tolist())):
            tit = dataset.title[i].split(' ')
            st = ''
            for t in tit:
                k = morph.parse(t)[0]
                st += k.normal_form
            if slovo in st:
                frame = frame.append(dataset.iloc[[i]])
    frame.drop_duplicates(keep=False, inplace=True)
    frame = frame.reset_index(drop=True)
    frame['coord'] = ''
    frame['dist'] = ''
    frame['rate'] = ''

    for i in range(0, len(frame.index)):
        frame['coord'][i] = fetch_coordinates(apikey, frame['location'][i])
        frame['dist'][i] = geodesic(center, frame['coord'][i], ellipsoid='WGS-84').km
        if frame['price'][i] != 'None':
            frame['rate'][i] = 4.3 * frame['dist'][i] + float(frame['price'][i])
        else:
            frame['rate'][i] = 1000000
    frame = frame.sort_values('rate')
    return frame

In [83]:
search('известь', 'Москва')

,title,price,izmer,location,seller,seller_link,date,link,coord,dist,rate
1,Известь гашеная,16.00,за Килограмм,"Рязанская область, Рязань",Станислав Юрьевич Наумов,https://agroru.com/pages/329536/,12.11.2020,https://agroru.com/doska/izvest-gashenaya-2430...,"(39.736375, 54.629216)",254.22,1109.15
2,"Известь негашеная ГОСТ (комовая, дробленая, мо...",2800.00,за Тонну,"Р-ка Башкортостан, Уфа",Урал,https://agroru.com/pages/51020/,14.05.2020,https://agroru.com/doska/izvest-negashenaya-go...,"(55.958727, 54.735147)",2039.79,11571.1
0,Известь негашеная ГОСТ,4800.00,за Тонну,"Р-ка Башкортостан, Уфа",Урал Робертович Гиниятуллин,https://agroru.com/pages/84613/,12.12.2020,https://agroru.com/doska/izvest-negashenaya-go...,"(55.958727, 54.735147)",2039.79,13571.1
5,"Известь негашеная комовая, известь гидратная (...",7000.00,за Тонну,"Р-ка Татарстан, Набережные Челны",Евгений,https://agroru.com/pages/188050/,NaN,https://agroru.com/doska/izvest-negashenaya-ko...,"(52.407031, 55.741004)",1643.04,14065.1
3,Известь не гашенная,6300.00,за Тонну,"Свердловская область, Нижний Тагил",Андрей,https://agroru.com/pages/229886/,NaN,https://agroru.com/doska/izvest-ne-gashennaya-...,"(59.972211, 57.907605)",2490.1,17007.4
4,Известь пушонка,None,None,"Украина, Черкассы",Володимир,https://agroru.com/pages/122016/,NaN,https://agroru.com/doska/izvest-pushonka-90383...,"(32.059759, 49.444433)",844.42,1000000
